In [1]:
using Interact
using CoordinateTransformations
using LCMGL
using ForwardDiff
using RigidBodyDynamics
using DataFrames

In [2]:
import Flash
reload("Flash")

adding: /home/rdeits/locomotion/explorations/point-cloud-signed-distance/packages/v0.5/DrakeVisualizer/src/lcmtypes to the python path


In [3]:
model = Flash.Models.beanbag()
state = Flash.ManipulatorState(model)
Flash.draw(state);

In [4]:
geometry = collect(values(model.surfaces[1].geometries))[1]
@manipulate for x = linspace(-1, 1, 51)
    state.deformations[geometry][1][:] = [x, 0, 0]
    Flash.draw(state)
end;

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=0),"x",0.0,linspace(-1.0,1.0,51),".3f",true)

In [5]:
# Construct the sensor and draw its view rays for debugging
sensor = Flash.DepthSensors.Kinect(80,80);
camera_origin = [0., 0, 8]
camera_tform = compose(Translation(camera_origin), LinearMap(AngleAxis(pi, 1, 0, 0)))
Flash.DepthSensors.draw_rays(sensor, camera_tform)

In [6]:
true_state = Flash.ManipulatorState(model)
true_state.mechanism_state.q[5:7] = 2 * rand(3)
true_state.deformation_data[:] = 0.5 .* (rand(Float64, length(true_state.deformation_data)) .- 0.5)
Flash.draw(true_state);

In [7]:
# Now we can raycast that true state to get a set of sensed points:
sensed_points = Flash.DepthSensors.raycast(true_state, sensor, camera_tform)
Flash.DepthSensors.draw_points(sensed_points)

cost = Flash.CostFunctor(model, sensed_points)

wrapped_cost = x -> begin
    x[1:4] = x[1:4] ./ norm(x[1:4])
    c = cost(x)
    Flash.draw(get(cost.state), false)
    c
end

(::#16) (generic function with 1 method)

In [8]:
cost_and_gradient! = Flash.CostAndGradientFunctor(wrapped_cost)

(::#29) (generic function with 1 method)

In [9]:
trials = []

function test_gradient_descent()
    for i = 1:1
        xs = []

        x_estimated = Flash.flatten(Flash.ManipulatorState(model))
        g = zeros(x_estimated)

        for i = 1:500
            push!(xs, x_estimated)

            # Evaluate the gradient of the point values w.r.t. the
            # joint angles:
            ci = cost_and_gradient!(g, x_estimated)

            if any(isnan, g)
                @show x_estimated
                error("NaN in gradient")
            end

            # Descend the gradient
            x_estimated -= 0.002 * g
            
            # TODO: generalize this
            x_estimated[1:4] = x_estimated[1:4] / norm(x_estimated[1:4])
        end
        push!(trials, DataFrame(x=xs))
    end
end

test_gradient_descent (generic function with 1 method)

In [10]:
test_gradient_descent()
state = Flash.ManipulatorState(model)
Flash.unflatten!(state, trials[1][:x][end])
Flash.draw(state)

DrakeVisualizer.Visualizer(DrakeVisualizer.Robot(DrakeVisualizer.Link[DrakeVisualizer.Link(DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{CoordinateTransformations.IdentityTransformation,GeometryTypes.HomogenousMesh{FixedSizeArrays.Point{3,Float64},GeometryTypes.Face{3,Int64,0},Void,Void,Void,Void,Void}}(HomogenousMesh(
    vertices: 6398xFixedSizeArrays.Point{3,Float64},     faces: 12792xGeometryTypes.Face{3,Int64,0}, )
,CoordinateTransformations.IdentityTransformation(),RGBA{Float64}(1.0,0.0,0.0,0.5))],"skin_1")]),1,PyLCM.LCM(PyObject <LCM object at 0x7fde3f814510>))

In [11]:
using Ipopt
using SimpleGradientDescent

In [12]:
state = Flash.ManipulatorState(model)
Flash.draw(state)
x0 = Flash.flatten(state)
num_vars = length(x0)
solver = IpoptSolver(print_level=2)

cost_functor = wrapped_cost
gradient_functor = Flash.CostAndGradientFunctor(Flash.CostFunctor(model, sensed_points))
m = getmodel(GradientDescent(cost_functor, gradient_functor), num_vars, solver)
setwarmstart!(m, x0);

In [13]:
optimize!(m)
x = getsolution(m)
state = Flash.ManipulatorState(model)
Flash.unflatten!(state, x)
Flash.draw(state)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************



DrakeVisualizer.Visualizer(DrakeVisualizer.Robot(DrakeVisualizer.Link[DrakeVisualizer.Link(DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{CoordinateTransformations.IdentityTransformation,GeometryTypes.HomogenousMesh{FixedSizeArrays.Point{3,Float64},GeometryTypes.Face{3,Int64,0},Void,Void,Void,Void,Void}}(HomogenousMesh(
    vertices: 6402xFixedSizeArrays.Point{3,Float64},     faces: 12800xGeometryTypes.Face{3,Int64,0}, )
,CoordinateTransformations.IdentityTransformation(),RGBA{Float64}(1.0,0.0,0.0,0.5))],"skin_1")]),1,PyLCM.LCM(PyObject <LCM object at 0x7fde3f8146c0>))